In [1]:
#import libraries

import requests
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    certifi-2018.8.24          |        py36_1001         139 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         220 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0        conda-forge
    geopy:         1.17.0-py_0      conda-forge

The following packages will be UPDATED:

    certifi:       2018.8.24-py36_1 conda-forge --> 2018.8.24-py36_1001 conda-forge


geopy-1.17.0         | 49 KB     | ##################################### | 100% 
geog

In [2]:
!pip install beautifulsoup4

distributed 1.21.8 requires msgpack, which is not installed.


In [3]:
!pip install lxml

distributed 1.21.8 requires msgpack, which is not installed.


In [4]:
!pip install html5lib

distributed 1.21.8 requires msgpack, which is not installed.


In [5]:
!pip install requests

distributed 1.21.8 requires msgpack, which is not installed.


In [6]:
from bs4 import BeautifulSoup

In [7]:
#Assign website link to a variable
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [8]:
#Use the function get from the requests library to download the Wikipedia page using the wikipedia_link as an argument. Assign the object to the variable postal_code_page.

#hint: requests.get()
source = requests.get(wikipedia_link).text
soup= BeautifulSoup(source,'lxml')
#print(soup.prettify())

In [9]:
table = soup.table
#print(table)
postheader = table.find_all('tr')
#print(postheader)
 

In [10]:
! pip install tabulate

    100% |████████████████████████████████| 51kB 12.1MB/s 
  Running setup.py bdist_wheel for tabulate ... done
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/2a/85/33/2f6da85d5f10614cbe5a625eab3b3aebfdf43e7b857f25f829
Successfully built tabulate
distributed 1.21.8 requires msgpack, which is not installed.


Get the data into a dataframe "DATA"

In [12]:
from tabulate import tabulate
table = soup.find_all('table')[0] 
df_list = pd.read_html(str(table))
#df_list[0]
type(df_list)
#print( tabulate(df_list[0], headers='keys', tablefmt='psql') )

data = pd.DataFrame(df_list[0])
type(data)
#data # 290*3

new_header = data.iloc[0] #grab the first row for the header
data = data[1:] #take the data less the header row
data.columns = new_header #set the header row as the df header

type(data) #289*3
#data

pandas.core.frame.DataFrame

Clean up data where data is NOT ASSIGNED

In [13]:
data = data[data.Borough != 'Not assigned']
data.head() #212*3

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


Massage Neighbouhood data to not to have NOT ASSIGNED value

In [14]:
#data['Neighbourhood'].loc[data['Neighbourhood'] == 'Not assigned'] = data[data.Borough]
#data.loc[data['Neighbourhood'] == 'Not assigned','Neighbourhood'] = data.loc[data[Borough]]
data['Neighbourhood'] = data.apply(
    lambda row: row['Borough'] if (row['Neighbourhood']=='Not assigned') else row['Neighbourhood'],
    axis=1
)
data.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


Grouping the records with the same postal code and have concatenated Neighbourhood values

In [15]:
data_group = data.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
data_group.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood]],Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
data_group.shape

(103, 3)

In [ ]:
Get Latitude and Longitude values for each neighbourhood

In [17]:

for index, row in data_group.iterrows():
    address_1 = row['Neighbourhood'] 
    address_2 = address_1.split(',')[-1]
    address_3 = address_2+","+"Ontario,Canada"
    #print(address_3) #-- It worked

In [18]:
column_names = ['Latitude', 'Longitude'] 
n_hood = pd.DataFrame(columns=column_names)
n_hood.shape

(0, 2)

In [19]:


for index, row in data_group.iterrows():
    try:
        address_1 = row['Neighbourhood'] 
        address_2 = address_1.split(',')[-1]
        address = address_2+","+"Ontario,Canada"
        geolocator = Nominatim()
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        #print(row['Borough'],address, latitude, longitude)
        n_hood = n_hood.append({'Latitude': latitude,'Longitude': longitude}, ignore_index=True)
        n_hood
        pass
    except ValueError as error_message:
        print("Error")
    except AttributeError:
        #print("Problem with data or cannot Geocode.")
        address_3 = row['Borough']
        address = address_3+","+"Ontario,Canada"
        geolocator = Nominatim()
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        #print(address, latitude, longitude)
        n_hood = n_hood.append({'Latitude': latitude,'Longitude': longitude}, ignore_index=True)
       # print(row['Borough'],address, latitude, longitude)
        n_hood
        pass



/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [25]:
n_hood.head()


,Latitude,Longitude
0,43.809196,-79.221701
1,43.775504,-79.134976
2,43.768914,-79.187291
3,43.759824,-79.225291
4,43.756467,-79.226692


Concatenate the latitude and longitude values to the main dataframe to complete the results

In [24]:
df = pd.concat([data_group, n_hood[['Latitude', 'Longitude']]], axis=1)
df.shape
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.809196,-79.221701
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.775504,-79.134976
2,M1E,Scarborough,"Guildwood]],Morningside,West Hill",43.768914,-79.187291
3,M1G,Scarborough,Woburn,43.759824,-79.225291
4,M1H,Scarborough,Cedarbrae,43.756467,-79.226692


In [32]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)
df.shape
len(df)

The dataframe has 11 boroughs and 103 neighborhoods.


103

In [27]:
#address = 'New York City, NY'
#address = 'Scarborough,Rouge,Malvern,Ontario,Canada'
address = 'Toronto,Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of New York City are 43.653963, -79.387207.


EXPLORE TORONTO

In [28]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


n_hood_1 = df
n_hood_1.head()
n_hood_2 = n_hood_1.drop('Postcode', axis=1)
len(n_hood_2)
n_hood_2.head()

In [ ]:
Explore Scarborough

In [38]:
Ontario_data = n_hood_2[n_hood_2['Borough'] == 'Scarborough'].reset_index(drop=True)
Ontario_data.head()
Ontario_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Scarborough,"Rouge,Malvern",43.809196,-79.221701
1,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.775504,-79.134976
2,Scarborough,"Guildwood]],Morningside,West Hill",43.768914,-79.187291
3,Scarborough,Woburn,43.759824,-79.225291
4,Scarborough,Cedarbrae,43.756467,-79.226692


In [36]:
address = 'Scarborough, Toronto,Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Manhattan are 43.773077, -79.257774.


In [41]:
# create map of Manhattan using latitude and longitude values
map_Scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Ontario_data['Latitude'], Ontario_data['Longitude'], Ontario_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough

In [42]:
CLIENT_ID = 'your-client-ID' # your Foursquare ID
CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
VERSION = '20180924' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your-client-ID
CLIENT_SECRET:your-client-secret


In [43]:
Ontario_data.loc[0, 'Neighbourhood']

'Rouge,Malvern'

In [44]:
neighborhood_latitude = Ontario_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Ontario_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Ontario_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.8091955, -79.2217008.


In [45]:

CLIENT_ID = 'EV4PLNEYRE4D43TC2ZA5IXZVHOJFLMIUW1YGAGZ1OUGYLKTW'
CLIENT_SECRET = '3OYWIAFIWRXE4GWJFFRWINC5GXXFPOS3RWR3HNPHRAWONUP1'
VERSION = '20180922'
radius = 1000
LIMIT = 500

#create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=EV4PLNEYRE4D43TC2ZA5IXZVHOJFLMIUW1YGAGZ1OUGYLKTW&client_secret=3OYWIAFIWRXE4GWJFFRWINC5GXXFPOS3RWR3HNPHRAWONUP1&v=20180922&ll=43.8091955,-79.2217008&radius=1000&limit=500'

In [46]:
results = requests.get(url).json()
#results

In [48]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [50]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()
len(nearby_venues)

15

In [51]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

15 venues were returned by Foursquare.


In [52]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [54]:
Ontario_venues = getNearbyVenues(names=Ontario_data['Neighbourhood'],
                                   latitudes=Ontario_data['Latitude'],
                                   longitudes=Ontario_data['Longitude']
                                  )


Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood]],Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West,Steeles West
Upper Rouge


In [55]:
print(Ontario_venues.shape)
Ontario_venues.head()

(181, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.809196,-79.221701,Shoppers Drug Mart,43.809202,-79.223320,Pharmacy
1,"Rouge,Malvern",43.809196,-79.221701,Subway,43.806805,-79.222515,Sandwich Place
2,"Rouge,Malvern",43.809196,-79.221701,Pizza Hut,43.808326,-79.220616,Pizza Place
3,"Rouge,Malvern",43.809196,-79.221701,Pizza Pizza,43.806613,-79.221243,Pizza Place
4,"Rouge,Malvern",43.809196,-79.221701,Shoppers Drug Mart,43.806489,-79.223024,Pharmacy


In [56]:
Ontario_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,11,11,11,11,11,11
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",1,1,1,1,1,1
"Birch Cliff,Cliffside West",10,10,10,10,10,10
Cedarbrae,20,20,20,20,20,20
"Clairlea,Golden Mile,Oakridge",5,5,5,5,5,5
"Clarks Corners,Sullivan,Tam O'Shanter",13,13,13,13,13,13
"Cliffcrest,Cliffside,Scarborough Village West",8,8,8,8,8,8
"Dorset Park,Scarborough Town Centre,Wexford Heights",15,15,15,15,15,15
"East Birchmount Park,Ionview,Kennedy Park",4,4,4,4,4,4


In [57]:
print('There are {} uniques categories.'.format(len(Ontario_venues['Venue Category'].unique())))

There are 65 uniques categories.


In [58]:
# one hot encoding
Ontario_onehot = pd.get_dummies(Ontario_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Ontario_onehot['Neighborhood'] = Ontario_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Ontario_onehot.columns[-1]] + list(Ontario_onehot.columns[:-1])
Ontario_onehot = Ontario_onehot[fixed_columns]

Ontario_onehot.head()

,Neighborhood,African Restaurant,Asian Restaurant,Auto Workshop,Bakery,Bank,Bar,Beer Store,Big Box Store,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Café,Chinese Restaurant,Coffee Shop,Convenience Store,Discount Store,Electronics Store,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Fried Chicken Joint,Furniture / Home Store,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hong Kong Restaurant,Hookah Bar,Ice Cream Shop,Korean Restaurant,Liquor Store,Market,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Optical Shop,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Sports Bar,Supermarket,Thai Restaurant,Toy / Game Store,Trail,Train Station,Video Game Store,Vietnamese Restaurant
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [59]:
Ontario_onehot.shape

(181, 66)

In [60]:
Ontario_grouped = Ontario_onehot.groupby('Neighborhood').mean().reset_index()
Ontario_grouped

,Neighborhood,African Restaurant,Asian Restaurant,Auto Workshop,Bakery,Bank,Bar,Beer Store,Big Box Store,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Café,Chinese Restaurant,Coffee Shop,Convenience Store,Discount Store,Electronics Store,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Fried Chicken Joint,Furniture / Home Store,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hong Kong Restaurant,Hookah Bar,Ice Cream Shop,Korean Restaurant,Liquor Store,Market,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Optical Shop,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Sports Bar,Supermarket,Thai Restaurant,Toy / Game Store,Trail,Train Station,Video Game Store,Vietnamese Restaurant
0,Agincourt,0.000000,0.181818,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.272727,0.090909,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.0,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.090909,0.00,0.000000
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,1.0,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000
2,"Birch Cliff,Cliffside West",0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.100000,0.000000,0.200000,0.0,0.100,0.000000,0.0,0.000000,0.100000,0.000000,0.000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000
3,Cedarbrae,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.050000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.050000,0.000000,0.150000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.050000,0.050000,0.000000,0.000000,0.000000,0.0,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.050000,0.050000,0.000000,0.050000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.050,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.050000,0.0,0.000000,0.05,0.000000
4,"Clairlea,Golden Mile,Oakridge",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.200000,0.000000,0.000000,0.0,0.000,0.000000,0.2,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000
5,"Clarks Corners,Sullivan,Tam O'Shanter",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.076923,0.153846,0.000000,0.00000

In [61]:
Ontario_grouped.shape

(17, 66)

In [62]:
num_top_venues = 5

for hood in Ontario_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Ontario_grouped[Ontario_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                venue  freq
0  Chinese Restaurant  0.27
1    Asian Restaurant  0.18
2   Electronics Store  0.09
3       Train Station  0.09
4   Korean Restaurant  0.09


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                venue  freq
0          Playground   1.0
1  African Restaurant   0.0
2   Korean Restaurant   0.0
3              Market   0.0
4      Medical Center   0.0


----Birch Cliff,Cliffside West----
            venue  freq
0     Pizza Place   0.2
1   Grocery Store   0.1
2  Breakfast Spot   0.1
3             Pub   0.1
4     Coffee Shop   0.1


----Cedarbrae----
                           venue  freq
0           Fast Food Restaurant  0.15
1         Furniture / Home Store  0.10
2                    Coffee Shop  0.10
3  Paper / Office Supplies Store  0.05
4                           Park  0.05


----Clairlea,Golden Mile,Oakridge----
               venue  freq
0     Ice Cream Shop   0.2
1           Bus Stop   0.2
2               Park   0.2

In [63]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [65]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Ontario_grouped['Neighborhood']

for ind in np.arange(Ontario_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Ontario_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()
len(neighborhoods_venues_sorted)

17

In [66]:
Ontario_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Scarborough,"Rouge,Malvern",43.809196,-79.221701
1,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.775504,-79.134976
2,Scarborough,"Guildwood]],Morningside,West Hill",43.768914,-79.187291
3,Scarborough,Woburn,43.759824,-79.225291
4,Scarborough,Cedarbrae,43.756467,-79.226692


In [69]:
Ontario_grouped.head()

,Neighborhood,African Restaurant,Asian Restaurant,Auto Workshop,Bakery,Bank,Bar,Beer Store,Big Box Store,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Café,Chinese Restaurant,Coffee Shop,Convenience Store,Discount Store,Electronics Store,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Fried Chicken Joint,Furniture / Home Store,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hong Kong Restaurant,Hookah Bar,Ice Cream Shop,Korean Restaurant,Liquor Store,Market,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Optical Shop,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Sports Bar,Supermarket,Thai Restaurant,Toy / Game Store,Trail,Train Station,Video Game Store,Vietnamese Restaurant
0,Agincourt,0.0,0.181818,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.272727,0.090909,0.0,0.00,0.090909,0.00,0.0,0.0,0.0,0.090909,0.0,0.0,0.0,0.00,0.00,0.0,0.090909,0.0,0.0,0.090909,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.090909,0.00,0.0
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.00,0.0
2,"Birch Cliff,Cliffside West",0.0,0.000000,0.1,0.0,0.0,0.0,0.00,0.00,0.1,0.0,0.0,0.0,0.0,0.0,0.000000,0.100000,0.0,0.00,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.10,0.00,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.10,0.0,0.20,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.00,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.00,0.0,0.000000,0.00,0.0
3,Cedarbrae,0.0,0.000000,0.0,0.0,0.0,0.0,0.05,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.100000,0.0,0.05,0.000000,0.15,0.0,0.0,0.0,0.000000,0.0,0.1,0.0,0.05,0.05,0.0,0.000000,0.0,0.0,0.000000,0.05,0.0,0.0,0.0,0.0,0.0,0.05,0.05,0.05,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.000000,0.05,0.0
4,"Clairlea,Golden Mile,Oakridge",0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.2,0.0,0.000000,0.000000,0.2,0.00,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.0,0.2,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.20,0.0,0.00,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.00,0.0


In [67]:
# set number of clusters
kclusters = 5

Ontario_grouped_clustering = Ontario_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Ontario_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 0, 0, 0, 0, 0, 4, 0], dtype=int32)

In [72]:
Ontario_merged = Ontario_data

# add clustering labels
Ontario_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Ontario_merged = Ontario_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

Ontario_merged.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Rouge,Malvern",43.809196,-79.221701,0,Pharmacy,Pizza Place,Skating Rink,Grocery Store,Gym / Fitness Center,Park,Sandwich Place,Fast Food Restaurant,Bubble Tea Shop,Discount Store
1,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.775504,-79.134976,1,Park,Vietnamese Restaurant,Convenience Store,Gym / Fitness Center,Gym,Grocery Store,Greek Restaurant,Furniture / Home Store,Fried Chicken Joint,Food Court
2,Scarborough,"Guildwood]],Morningside,West Hill",43.768914,-79.187291,0,Coffee Shop,Pizza Place,Fast Food Restaurant,Breakfast Spot,Filipino Restaurant,Sandwich Place,Discount Store,Medical Center,Chinese Restaurant,Pharmacy
3,Scarborough,Woburn,43.759824,-79.225291,0,Fast Food Restaurant,Coffee Shop,Grocery Store,Bus Station,Furniture / Home Store,Paper / Office Supplies Store,Pharmacy,Pizza Place,Discount Store,Vietnamese Restaurant
4,Scarborough,Cedarbrae,43.756467,-79.226692,0,Fast Food Restaurant,Coffee Shop,Furniture / Home Store,Shopping Mall,Video Game Store,Liquor Store,Optical Shop,Paper / Office Supplies Store,Park,Discount Store


In [75]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Ontario_merged['Latitude'], Ontario_merged['Longitude'], Ontario_merged['Neighbourhood'], Ontario_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

 Cluster 1 : Seems to be the most happening place with lots of restaurants, bus facilities, supermarkets etc.,

In [79]:
Ontario_merged.loc[Ontario_merged['Cluster Labels'] == 0, Ontario_merged.columns[[1] + list(range(5, Ontario_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge,Malvern",Pharmacy,Pizza Place,Skating Rink,Grocery Store,Gym / Fitness Center,Park,Sandwich Place,Fast Food Restaurant,Bubble Tea Shop,Discount Store
2,"Guildwood]],Morningside,West Hill",Coffee Shop,Pizza Place,Fast Food Restaurant,Breakfast Spot,Filipino Restaurant,Sandwich Place,Discount Store,Medical Center,Chinese Restaurant,Pharmacy
3,Woburn,Fast Food Restaurant,Coffee Shop,Grocery Store,Bus Station,Furniture / Home Store,Paper / Office Supplies Store,Pharmacy,Pizza Place,Discount Store,Vietnamese Restaurant
4,Cedarbrae,Fast Food Restaurant,Coffee Shop,Furniture / Home Store,Shopping Mall,Video Game Store,Liquor Store,Optical Shop,Paper / Office Supplies Store,Park,Discount Store
5,Scarborough Village,Coffee Shop,Supermarket,Fast Food Restaurant,Shopping Mall,Pub,Chinese Restaurant,Gym,Furniture / Home Store,Fried Chicken Joint,Food Court
6,"East Birchmount Park,Ionview,Kennedy Park",Fast Food Restaurant,Asian Restaurant,Grocery Store,Vietnamese Restaurant,Convenience Store,Gym / Fitness Center,Gym,Greek Restaurant,Furniture / Home Store,Fried Chicken Joint
7,"Clairlea,Golden Mile,Oakridge",Ice Cream Shop,Bus Stop,Park,Convenience Store,Restaurant,Bank,Bar,Gym,Grocery Store,Greek Restaurant
9,"Birch Cliff,Cliffside West",Pizza Place,Snack Place,Coffee Shop,Pub,Park,Sandwich Place,Breakfast Spot,Auto Workshop,Grocery Store,Food Court
12,Agincourt,Chinese Restaurant,Asian Restaurant,Food Court,Hong Kong Restaurant,Electronics Store,Coffee Shop,Korean Restaurant,Train Station,Grocery Store,Greek Restaurant
13,"Clarks Corners,Sullivan,Tam O'Shanter",Coffee Shop,Pharmacy,Pizza Place,Thai Restaurant,Market,Fried Chicken Joint,Fast Food Restaurant,Bus Stop,Grocery Store,Chinese Restaurant


Cluster 2: Must be a place with lot of greenery to relax and have some food

In [80]:
Ontario_merged.loc[Ontario_merged['Cluster Labels'] == 1, Ontario_merged.columns[[1] + list(range(5, Ontario_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek,Rouge Hill,Port Union",Park,Vietnamese Restaurant,Convenience Store,Gym / Fitness Center,Gym,Grocery Store,Greek Restaurant,Furniture / Home Store,Fried Chicken Joint,Food Court
11,"Maryvale,Wexford",Pizza Place,Middle Eastern Restaurant,Grocery Store,Convenience Store,Pharmacy,Korean Restaurant,Hookah Bar,Fish Market,Vietnamese Restaurant,Rental Car Location


Clutser 3: Must be a place for kids to assemble and play basically an active zone 

In [81]:
Ontario_merged.loc[Ontario_merged['Cluster Labels'] == 2, Ontario_merged.columns[[1] + list(range(5, Ontario_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,"L'Amoreaux West,Steeles West",Playground,Vietnamese Restaurant,Convenience Store,Gym / Fitness Center,Gym,Grocery Store,Greek Restaurant,Furniture / Home Store,Fried Chicken Joint,Food Court


Cluster 4: Good for eating and shopping

In [82]:
Ontario_merged.loc[Ontario_merged['Cluster Labels'] == 3, Ontario_merged.columns[[1] + list(range(5, Ontario_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,"Dorset Park,Scarborough Town Centre,Wexford He...",Middle Eastern Restaurant,Grocery Store,Pizza Place,Mediterranean Restaurant,Bus Station,Hookah Bar,Fast Food Restaurant,Chinese Restaurant,Supermarket,Bar


Cluster 5: Must be a place to hangout and do some shopping

In [83]:
Ontario_merged.loc[Ontario_merged['Cluster Labels'] == 4, Ontario_merged.columns[[1] + list(range(5, Ontario_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Cliffcrest,Cliffside,Scarborough Village West",Coffee Shop,Supermarket,Fast Food Restaurant,Shopping Mall,Pub,Chinese Restaurant,Gym,Furniture / Home Store,Fried Chicken Joint,Food Court
